In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time 
import sys
from tqdm import tqdm
from skimage import io
from skimage import measure
from scipy.ndimage import morphology
from at_synapse_detection import dataAccess as da

In [ ]:
from at_synapse_detection import SynapseDetection as syn

In [ ]:
%matplotlib notebook

In [ ]:
psd_fn = 'collman_M247514_Rorb_1_Site3Align2_LENS_Session1_CROP_0_438_0_460_0_49/collman_M247514_Rorb_1_Site3Align2_LENS_Session1_CROP_0_438_0_460_0_49_PSD95_0.tiff'

In [ ]:
synapsin_fn = 'collman_M247514_Rorb_1_Site3Align2_LENS_Session1_CROP_0_438_0_460_0_49/collman_M247514_Rorb_1_Site3Align2_LENS_Session1_CROP_0_438_0_460_0_49_synapsin_0.tiff'

In [ ]:
psd_vol = da.imreadtiff(psd_fn)
synapsin_vol = da.imreadtiff(synapsin_fn)

In [ ]:
synaptic_volumes = {'presynaptic': [synapsin_vol], 'postsynaptic': [psd_vol]}
min_num_of_slices = 1
# Create query
query = {'preIF': ['red_channel'], 'preIF_z': [min_num_of_slices],
         'postIF': ['green_channel'], 'postIF_z': [min_num_of_slices],
         'punctumSize': 2}

In [ ]:
resultVol = syn.getSynapseDetections(synaptic_volumes, query)

In [ ]:
resultVol.shape

In [ ]:
thresh  =  0.9
label_vol = measure.label(resultVol > thresh)
stats = measure.regionprops(label_vol)

In [ ]:
len(stats)


In [ ]:
bw_vol  = resultVol > thresh

In [ ]:
SE = morphology.generate_binary_structure(2, 2)

In [ ]:
SE

In [ ]:
bw_vol_dilate = np.copy(bw_vol)

for z in range(0, bw_vol.shape[2]): 
    bw_vol_dilate[:, :, z] = morphology.binary_dilation(bw_vol[:, :, z], SE)

In [ ]:
plt.figure()
plt.imshow(bw_vol_dilate[:, :, 12])

In [ ]:
np.sum(bw_vol_dilate)

In [ ]:
np.sum(bw_vol)

In [ ]:
maskvol = np.zeros((460, 438, 49))

In [ ]:
#create mask 

global_minx = 152
global_maxx = 288

global_miny = 149
global_maxy = 317

In [ ]:
maskvol[global_minx:global_maxx, global_miny:global_maxy, :] = 1 

In [ ]:
plt.figure()
plt.imshow(maskvol[:, :, 13])

In [ ]:
masked_detections_vol = maskvol*bw_vol_dilate

In [ ]:
plt.figure()
plt.imshow(masked_detections_vol[:, :, 11])

In [ ]:
# Run regionprops
label_vol = measure.label(masked_detections_vol)
detections = measure.regionprops(label_vol)


In [ ]:
len(detections)

In [ ]:
uniquedetections = np.unique(label_vol)

In [ ]:
type(uniquedetections)

In [ ]:
uniquedetections = np.delete(uniquedetections, [0])

In [ ]:
uniquedetections

In [ ]:
lm_labels_dict = np.load('lm_labels_dict.npy').item()

In [ ]:
keylist = lm_labels_dict.keys()

In [ ]:
synapse_detection = detections[4]

In [ ]:
synapse_label = lm_labels_dict[4]

In [ ]:
len(synapse_label)

In [ ]:
def make_label_vol(synapse_label, shape): 
    labelvol = np.zeros(shape, dtype=np.int64)
    for n in range(0, synapse_label.shape[0]): 
        labelvol[synapse_label[n][0], synapse_label[n][1], synapse_label[n][2]] = -1
    
    return labelvol

In [ ]:
synapse_label = lm_labels_dict[2]
shape = (460, 438, 49)
single_label_vol = make_label_vol(synapse_label, shape) 

In [ ]:
np.unique(single_label_vol)

In [ ]:
single_label_vol.dtype

In [ ]:
detection_vol.dtype

In [ ]:
detection_vol = np.copy(label_vol)
test_vol = single_label_vol*detection_vol

In [ ]:
unique_vals = np.unique(test_vol)

In [ ]:
np.unique(single_label_vol)

In [ ]:
# Determine overlap

In [ ]:
output_mat

In [ ]:
print(unique_vals)

In [ ]:
index = np.argwhere(unique_vals==0)
y = np.abs(np.delete(unique_vals, index))

In [ ]:
print(y)

In [ ]:
len([0])

In [ ]:
output_mat[1, y] = 1 

In [ ]:
output_mat[1, 200:220]

In [ ]:
shape = (460, 438, 49)

In [ ]:
output_mat = np.zeros((len(keylist), len(detections)))

In [ ]:
from tqdm import trange

In [ ]:
hit_count = 0
miss_count = 0
for label_ind, key in enumerate(lm_labels_dict.keys()): 
    
    synapse_label = lm_labels_dict[key]
    single_label_vol = make_label_vol(synapse_label, shape) 
    
    label_detection_vol = single_label_vol*detection_vol
    unique_vals = np.unique(label_detection_vol)
    
    if len(unique_vals) > 1: 
        index = np.argwhere(unique_vals==0)
        unique_vals = np.abs(np.delete(unique_vals, index))
        output_mat[label_ind, unique_vals] = 1 
        hit_count = hit_count + 1 
    else: 
        miss_count = miss_count + 1 
             

In [ ]:
print('Hit Count: ', hit_countnt)
print('Miss Count: 'miss_countcount)

In [ ]:
# Attempt 2 

In [ ]:
miss_count

In [ ]:
len(lm_labels_dict.keys())

In [ ]:
636.0+476